In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import torch
import torch.nn as nn
from pyDOE import lhs

from architectures.FourierFeatures import FourierFeatures
from architectures.MLP import MLP
from architectures.MultiTaskMLP import MultiTaskMLP
from training.Trainer import Trainer
from inference.PDESR import PDESR

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cuda


# Data Loading

In [ ]:
# Load data

data = scipy.io.loadmat('./Data/burgers_shock_mu_01_pi.mat')
x = data['x'].flatten()
t = data['t'].flatten()
usol = data['usol']

X, T = np.meshgrid(x, t, indexing='ij')
lower_bound = np.array([X.min(), T.min()])
upper_bound = np.array([X.max(), T.max()])

In [ ]:
# Initial conditions

xt_ic = np.hstack([X[:, 0][:, None], T[:, 0][:, None]])
u_ic = usol[:, 0][:, None]

xt_ic = torch.from_numpy(xt_ic).float().to(DEVICE)
u_ic = torch.from_numpy(u_ic).float().to(DEVICE)

In [ ]:
# Boundary conditions

xt_bc_left = np.hstack([X[0, :][:, None], T[0, :][:, None]])    # x=x_min
u_bc_left = usol[0, :][:, None]

xt_bc_right = np.hstack([X[-1, :][:, None], T[-1, :][:, None]]) # x=x_max
u_bc_right = usol[-1, :][:, None]

xt_bc = np.vstack([xt_bc_left, xt_bc_right])
u_bc = np.vstack([u_bc_left, u_bc_right])

xt_bc = torch.from_numpy(xt_bc).float().to(DEVICE)
u_bc = torch.from_numpy(u_bc).float().to(DEVICE)

In [ ]:
# Interior Points

mask_interior = (T != t.min()) & (X != x.min()) & (X != x.max())

X_flat = X[mask_interior][:, None]
T_flat = T[mask_interior][:, None]
U_flat = usol[mask_interior][:, None]

xt_data = torch.from_numpy(np.hstack([X_flat, T_flat])).float().to(DEVICE)
u_data = torch.from_numpy(U_flat).float().to(DEVICE)

In [ ]:
# Collocation points

N_col = 10_000
xt_col = lhs(n=2, samples=N_col)
xt_col[:,0] = 2 * xt_col[:,0] - 1
xt_col = torch.from_numpy(xt_col).float().to(DEVICE)

In [ ]:
xt_ic.shape, xt_bc.shape, xt_data.shape

(torch.Size([256, 2]), torch.Size([200, 2]), torch.Size([25146, 2]))

# Training

In [ ]:
# Trainer settings
lhs_deriv = 'u_t'
deriv_library = ['u_x', 'u_xx']
add_const_coeff = False
input_names = ['x', 't']
coeffs_input_names = ['x', 'u']
n_epochs=1000
sol_warmup_n_epochs=200
sol_update_per_epoch=5
coeffs_update_per_epoch=5
lambda_pde_warmup_epochs=500
max_lambda_pde=1.0
sol_warmup_loss_weighting=False
fit_loss_weighting=True
lambda_ic=1.0
lambda_bc=1.0
lambda_data=1.0
lambda_pde=1.0
lambda_alpha=0.9
verbosity=10

# Solution model settings
num_frequencies = 20
scale = 1.
input_dim = 2
sol_hidden_dims = [512, 512, 512, 512]
sol_output_dim = 1
activation = nn.GELU()

# Coefficients model settings
shared_hidden_dims = [512, 512, 512, 512]
head_num_layers = 2
n_coeffs = len(deriv_library) + int(add_const_coeff)

In [ ]:
sol_fourier_features = FourierFeatures(input_dim,
                                       num_frequencies=num_frequencies,
                                       scale=scale).to(dtype=torch.float32, device=DEVICE)
sol_model = MLP(input_dim=input_dim,
                hidden_dims=sol_hidden_dims,
                output_dim=sol_output_dim,
                activation=activation,
                fourier_features=sol_fourier_features).to(device=DEVICE)
sol_optimizer = torch.optim.Adam(sol_model.parameters(), lr=1e-4)
sol_lr_scheduler = None

In [ ]:
coeffs_fourier_features = None
coeffs_model = MultiTaskMLP(input_dim=input_dim,
                            shared_hidden_dims=shared_hidden_dims,
                            head_num_layers=head_num_layers,
                            n_coeffs=n_coeffs,
                            activation=activation,
                            fourier_features=coeffs_fourier_features).to(device=DEVICE)
coeffs_optimizer = torch.optim.Adam(coeffs_model.parameters(), lr=1e-4)
fit_lr_scheduler = None

In [ ]:
trainer = Trainer(lhs_deriv=lhs_deriv,
                  deriv_library=deriv_library,
                  add_const_coeff=add_const_coeff,
                  input_names=input_names,
                  coeffs_input_names=coeffs_input_names,
                  sol_optimizer=sol_optimizer,
                  coeffs_optimizer=coeffs_optimizer,
                  sol_lr_scheduler=sol_lr_scheduler,
                  fit_lr_scheduler=fit_lr_scheduler,
                  n_epochs=n_epochs,
                  sol_warmup_n_epochs=sol_warmup_n_epochs,
                  sol_update_per_epoch=sol_update_per_epoch,
                  coeffs_update_per_epoch=coeffs_update_per_epoch,
                  lambda_pde_warmup_epochs=lambda_pde_warmup_epochs,
                  max_lambda_pde=max_lambda_pde,
                  sol_warmup_loss_weighting=sol_warmup_loss_weighting,
                  fit_loss_weighting=fit_loss_weighting,
                  lambda_ic=lambda_ic,
                  lambda_bc=lambda_bc,
                  lambda_data=lambda_data,
                  lambda_pde=lambda_pde,
                  lambda_alpha=lambda_alpha,
                  verbosity=verbosity)

In [ ]:
res = trainer.fit(sol_model=sol_model,
                  coeffs_model=coeffs_model,
                  X_ic=xt_ic, U_ic=u_ic, X_bc=xt_bc, U_bc=u_bc, X_data=xt_data, U_data=u_data, X_col=xt_col)

[SolWarmup] Ep 10/200 Total=6.924988e-01, IC=3.633154e-01, BC=1.070078e-03, Data=3.281133e-01, L_IC=1.000000e+00, L_BC=1.000000e+00, L_data=1.000000e+00
[SolWarmup] Ep 20/200 Total=3.907560e-01, IC=1.453459e-01, BC=8.196167e-03, Data=2.372140e-01, L_IC=1.000000e+00, L_BC=1.000000e+00, L_data=1.000000e+00
[SolWarmup] Ep 30/200 Total=1.584845e-01, IC=8.436498e-03, BC=2.721835e-02, Data=1.228297e-01, L_IC=1.000000e+00, L_BC=1.000000e+00, L_data=1.000000e+00
[SolWarmup] Ep 40/200 Total=9.606323e-02, IC=1.707338e-02, BC=1.109786e-02, Data=6.789199e-02, L_IC=1.000000e+00, L_BC=1.000000e+00, L_data=1.000000e+00
[SolWarmup] Ep 50/200 Total=5.840504e-02, IC=5.986469e-03, BC=2.586452e-03, Data=4.983212e-02, L_IC=1.000000e+00, L_BC=1.000000e+00, L_data=1.000000e+00
[SolWarmup] Ep 60/200 Total=4.406445e-02, IC=1.356764e-03, BC=1.848752e-03, Data=4.085894e-02, L_IC=1.000000e+00, L_BC=1.000000e+00, L_data=1.000000e+00
[SolWarmup] Ep 70/200 Total=3.924567e-02, IC=1.683293e-03, BC=1.141225e-03, Data=3

# Inference

In [ ]:
pdesr = PDESR(trainer=trainer,
              niterations=100,
              early_stop_condition=(
                  "stop_if(loss, complexity) = loss < 1e-10 && complexity < 10"
                  ),
              timeout_in_seconds=5 * 60,
              maxsize=10,
              maxdepth=10,
              binary_operators=["*", "+", "-", "/"],
              unary_operators=["square", "cube", "exp", "cos", "sin"],
              constraints={
                  "/": (-1, 3),
                  "square": 3,
                  "cube": 3,
                  "exp": 3,
                  "sin": 3,
                  "cos": 3
                  },
              complexity_of_operators={"/": 2, "exp": 4, "cos": 2, "sin": 2},
              complexity_of_constants=1,
              model_selection='score',
              output_directory=None)

In [ ]:
terms_res = pdesr.fit(xt_data)


Fitting SR on u_x coefficient...


/usr/local/lib/python3.12/dist-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pysr/sr.py:2265: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://ai.damtp.cam.ac.uk/pysr/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enou


Expressions evaluated per second: 1.840e+01
Progress: 1 / 3100 total iterations (0.032%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           3.597e-01  0.000e+00  y = -0.0003464
6           3.166e-01  2.557e-02  y = sin(square(-0.44181)) + x
7           2.949e-01  7.087e-02  y = 1.8536 * ((x - u) - x)
9           2.213e-01  1.436e-01  y = (u - sin(0.87873 + 1.3709)) * -0.35372
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.830e+03
Progress: 73 / 3100 total iterations (2.355%)
═══════════════════════════════════════════════════════════════════════

[ Info: Final population:
[ Info: Results saved to:
/usr/local/lib/python3.12/dist-packages/pysr/sr.py:2217: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pysr/sr.py:2265: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://ai.damtp.cam.ac.uk/pysr/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.788e-01  0.000e+00  y = x
3           3.458e-04  3.346e+00  y = u * -0.97287
5           3.390e-04  9.867e-03  y = (u * -0.97289) + 0.0025988
6           3.054e-04  1.044e-01  y = (cube(u) * 0.041253) - u
7           1.833e-04  5.104e-01  y = ((x * -0.028388) + u) * -0.95815
9           1.768e-04  1.813e-02  y = ((x * 0.027181) + (u * -0.95819)) + 0.0025547
10          1.624e-04  8.505e-02  y = (u * -0.96634) + (cube(x + 0.13815) * 0.031775)
───────────────────────────────────────────────────────────────────────────────────────────────────

Finished SR on u_x coefficient!

Fitting SR on u_xx coefficient...
  - outputs/20251130_212316_zMql6m/hall_of_fame.csv


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 1.230e+02
Progress: 2 / 3100 total iterations (0.065%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.149e-07  0.000e+00  y = 0.0029369
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 5.040e+03
Progress: 85 / 3100 total iterations (2.742%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.149e-07  0.000e+

[ Info: Final population:
[ Info: Results saved to:



Expressions evaluated per second: 9.250e+03
Progress: 3063 / 3100 total iterations (98.806%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.149e-07  0.000e+00  y = 0.0029369
5           1.956e-07  2.419e-01  y = (x * 0.00098252) + 0.0029369
7           1.334e-07  1.914e-01  y = cos(x - 0.30255) * 0.0036173
8           9.729e-08  3.157e-01  y = (cube(x - 0.66806) * 0.00050912) + 0.0034262
10          8.014e-08  9.692e-02  y = (cube(x) * ((u * 0.005896) + 0.0014735)) + 0.003328
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Press 'q' and then <enter> to stop execution early.
──────────────────────────────────────────────────────

In [ ]:
terms_res['u_x'] # 0, 1, 2, 4, 6

{'complexity': {0: 1, 1: 3, 2: 5, 3: 6, 4: 7, 5: 9, 6: 10},
 'loss': {0: 0.27879986,
  1: 0.000345774,
  2: 0.0003390173,
  3: 0.00030539653,
  4: 0.0001833162,
  5: 0.00017678915,
  6: 0.00016237532},
 'equation': {0: 'x',
  1: 'u * -0.9728695',
  2: '(u * -0.97289103) + 0.0025988456',
  3: '(cube(u) * 0.0412528) - u',
  4: '((x * -0.028387887) + u) * -0.95815486',
  5: '((x * 0.027180633) + (u * -0.9581852)) + 0.002554741',
  6: '(u * -0.9663392) + (cube(x + 0.13814926) * 0.03177467)'},
 'score': {0: 0.0,
  1: 3.3462320366318954,
  2: 0.009867121962895771,
  3: 0.10444010795473704,
  4: 0.5103984999261755,
  5: 0.018127375551841112,
  6: 0.08504733380657209},
 'sympy_format': {0: x,
  1: u*(-0.9728695),
  2: 0.0025988456 + u*(-0.97289103),
  3: u**3*0.0412528 - u,
  4: (u + x*(-0.028387887))*(-0.95815486),
  5: u*(-0.9581852) + x*0.027180633 + 0.002554741,
  6: u*(-0.9663392) + (x + 0.13814926)**3*0.03177467},
 'lambda_format': {0: PySRFunction(X=>x),
  1: PySRFunction(X=>u*(-0.97286

In [ ]:
terms_res['u_xx']

{'complexity': {0: 1, 1: 5, 2: 7, 3: 8, 4: 10},
 'loss': {0: 5.1487285e-07,
  1: 1.9561713e-07,
  2: 1.3339898e-07,
  3: 9.7286986e-08,
  4: 8.0144275e-08},
 'equation': {0: '0.0029369364',
  1: '(x * 0.0009825161) + 0.0029369434',
  2: 'cos(x - 0.3025483) * 0.0036172771',
  3: '(cube(x - 0.66806114) * 0.0005091175) + 0.0034261819',
  4: '(cube(x) * ((u * 0.0058960365) + 0.0014734638)) + 0.003327989'},
 'score': {0: 0.0,
  1: 0.2419401616308722,
  2: 0.19140742158891982,
  3: 0.31567925831360744,
  4: 0.09691839051439036},
 'sympy_format': {0: 0.00293693640000000,
  1: x*0.0009825161 + 0.0029369434,
  2: cos(x - 1*0.3025483)*0.0036172771,
  3: (x - 1*0.66806114)**3*0.0005091175 + 0.0034261819,
  4: x**3*(u*0.0058960365 + 0.0014734638) + 0.003327989},
 'lambda_format': {0: PySRFunction(X=>0.00293693640000000),
  1: PySRFunction(X=>x*0.0009825161 + 0.0029369434),
  2: PySRFunction(X=>cos(x - 1*0.3025483)*0.0036172771),
  3: PySRFunction(X=>(x - 1*0.66806114)**3*0.0005091175 + 0.003426181